<a href="https://colab.research.google.com/github/Akshaypakhle10/ML/blob/master/Single_Layer_Perceptron_With_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TF basics

Tensors - Multi Dimensional Arrays, Flow - Data Flow graph

Core programs : 
1. Build a Computational Graph of Nodes.
2. Run the computational graph within a session.

In [0]:
import tensorflow as tf

node1 = tf.constant(2.0, tf.float32)
node2 = tf.constant(4.0)
#print(node1, node2)

Tensor("Const_1:0", shape=(), dtype=float32) Tensor("Const_2:0", shape=(), dtype=float32)


To actually evaluate the nodes, we must run a session. A session encapsulates the control and state of the TF runtime

In [0]:
sess = tf.Session()
print(sess.run([node1, node2]))

[2.0, 4.0]


It always produces a constant result, what if I want to change values?

In [0]:
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

d = tf.multiply(a,b)
e = tf.add(c,b)
f = tf.subtract(d,e)
sess = tf.Session()

outs = sess.run(f)
print(outs)

5


A graph can be parameterized to accept external ips as well
A placeholder is a promsie to provide a value later.

In [0]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

adder_node  = a+b
sess = tf.Session()

print(sess.run(adder_node, {a:[1,3], b:[2,4]}))

[3. 7.]


Variables? To add trainable parameters to a graph

In [0]:
w = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)

x = tf.placeholder(tf.float32)

linear_model = w*x + b
init = tf.global_variables_initializer()

sess = tf.Session()

sess.run(init)
print(sess.run(linear_model, {x:[1,2,3,4]}))

[0.         0.3        0.6        0.90000004]


A loss function measures how far apart the current model is from the provided data.

In [0]:
y = tf.placeholder(tf.float32)

square_deltas = tf.square(linear_model - y)

loss = tf.reduce_sum(square_deltas)

print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


This is too much loss, to reduce loss, tf provides variables to minimize loss function, simplest is Gradient Descent

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init)

for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})
  
print(sess.run([w,b]))

[array([-0.9999969], dtype=float32), array([0.9999908], dtype=float32)]


# AND Gate Implementation

In [0]:
import tensorflow as tf

T , F = 1., -1.

bias = 1.

train_in = [
    [T,T,bias],
    [T,F,bias],
    [F,T,bias],
    [F,F, bias],
]


train_out = [
    [T],
    [F],
    [F],
    [F],
]

w = tf.Variable(tf.random_normal([3,1]))
                
def step(x):
                is_greater = tf.greater(x,0)
                as_float = tf.cast(is_greater, float)
                doubled = tf.multiply(as_float, 2)
                return tf.subtract(doubled,1)
               
output = step(tf.matmul(train_in, w))
error = tf.subtract(train_out, output)
mse = tf.reduce_mean(tf.square(error))
                
delta = tf.matmul(train_in, error, transpose_a = True)
train = tf.assign(w, tf.add(w,delta))
                
sess = tf.Session()
sess.run(tf.initialize_all_variables())
               
err, target = 1, 0
epoch, max_epoch = 0, 10
while err>target and epoch<max_epoch:
                epoch +=1
                err, _ = sess.run([mse,train])
                print('epoch:', epoch, 'mse:',err)

epoch: 1 mse: 1.0
epoch: 2 mse: 2.0
epoch: 3 mse: 0.0


#OR Gate

In [0]:
T , F = 1. ,-1.
bias = 1.

train_in = [
    [T,T,bias],
    [T,F,bias],
    [F,T,bias],
    [F,F,bias],
]

train_out = [
    [T],
    [T],
    [T],
    [F],
]

w = tf.Variable(tf.random_normal([3,1]))

def step(x):
  is_greater = tf.greater(x,0)
  to_float = tf.cast(is_greater, float)
  doubled = tf.multiply(to_float,2)
  return tf.subtract(doubled,1)

output = step(tf.matmul(train_in,w))
error = tf.subtract(train_out,output)
mse = tf.reduce_mean(tf.square(error))

delta = tf.matmul(train_in, error, transpose_a=True)
train = tf.assign(w, tf.add(w,delta))

                
sess = tf.Session()
sess.run(tf.initialize_all_variables())
               
err, target = 1, 0
epoch, max_epoch = 0, 10
while err>target and epoch<max_epoch:
                epoch +=1
                err, _ = sess.run([mse,train])
                print('epoch:', epoch, 'mse:',err)

epoch: 1 mse: 3.0
epoch: 2 mse: 1.0
epoch: 3 mse: 0.0


Tensor("random_normal_23:0", shape=(3, 1), dtype=float32)


#MNIST 

In [0]:
# import tensorflow as tf
# mnist = tf.keras.datasets.mnist
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape = [None, 784])
y = tf.placeholder(tf.float32, shape = [None,10])

w = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

y_ = tf.matmul(x,w) + b

#cross entropy loss function

cross_entropy = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = y_) 
)

#train the model using optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict = {x: batch[0], y: batch[1]})

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict = {x:mnist.test.images, y: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.9158
